In [1]:
##vocab production script
#imports
import multiprocessing
from multiprocessing import Pool
import math, random, sys
import pickle
import argparse
from functools import partial
import torch
import numpy
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from hgraph import MolGraph, common_atom_vocab, PairVocab
import rdkit
import rdkit.Chem as Chem
import networkx as nx
from hgraph.chemutils import *
from hgraph.nnutils import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import SVG
import time
from rdkit.Chem import Draw
import itertools
from collections import Counter

# IPythonConsole.drawOptions.addAtomIndices = True
# IPythonConsole.molSize = 300,300

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'rdkit.Chem.Draw.IPythonConsole' has no attribute 'drawOptions'

In [273]:
# peptide = 'OC(C[C@@H](C(N[C@H](C(N[C@H](C(N[C@H](C(NCC(N[C@H](C(N[C@@]([H])([C@@H](C)CC)C(N[C@@H](CC(C)C)C(N[C@H](C(N[C@H](C(N[C@@H](CC1=CC=CC=C1)C(N2CCC[C@H]2C(N[C@H](C(N[C@@]([C@H](O)C)(C(N[C@H](C(N[C@H](C(N[C@H](C(N[C@H](C(N[C@H](C(O)=O)CSC3)=O)CSCC3=O)=O)C(C)C)=O)CC4=CC=C(O)C=C4)=O)CC5=CNC6=C5C=CC=C6)=O)[H])=O)CO)=O)=O)=O)CCSC)=O)CCCCN[H])=O)=O)=O)CC7=CNC=N7)=O)=O)CCC(N)=O)=O)CCC(O)=O)=O)CS[H])=O)NC([C@@H](NC([C@@H](NC([C@H](C)NC(CN)=O)=O)CCCNC(N)=N)=O)CC(N)=O)=O)=O'
#c term proline
# peptide = 'N[C@@H]([C@]([H])(O)C)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](C(C)C)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CCC(=O)O)C(=O)N[C@@H](CC1=CN=C-N1)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CO)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS3)C(=O)N1[C@@H](CCC1)C(=O)O'
#n term proline
# peptide = 'N1[C@@H](CCC1)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](CS3)C(=O)N[C@@H](C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CC(=CN2)C1=C2C=CC=C1)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CCCNC(=N)N)C(=O)O'
#random peptides
peptide = 'N[C@@H]([H])C(=O)N[C@@H](CSCC(C4)=O)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CCC(=O)N)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](C(C)C)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS4)C(=O)N1[C@@H](CCC1)C(=O)O'

peptide = get_mol(peptide)

In [274]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

mol_with_atom_index(peptide)

Draw.MolToFile(peptide, "peptide.png",size=(6000,6000))

In [275]:
#separately define proline, because the backbone creates double recognition
proline = 'O=CC1CCCN1'
proline_matches = peptide.GetSubstructMatches(Chem.MolFromSmiles(proline),useChirality=True)
proline_matches

((20, 19, 15, 16, 17, 18, 14), (96, 95, 91, 92, 93, 94, 90))

In [276]:
#find all amino acids
amino_acid_motif = 'NCC=O'
matches = peptide.GetSubstructMatches(Chem.MolFromSmiles(amino_acid_motif),useChirality=True)
matches

((0, 1, 2, 3),
 (4, 5, 12, 13),
 (14, 15, 19, 20),
 (21, 22, 28, 29),
 (30, 31, 34, 35),
 (36, 37, 42, 43),
 (44, 45, 49, 50),
 (51, 52, 59, 60),
 (61, 62, 71, 72),
 (73, 74, 82, 83),
 (84, 85, 88, 89),
 (90, 91, 95, 96))

In [277]:
#find matches that aren't prolines
matches_without_prolines = [match for match in matches if not any([x in np.hstack(proline_matches) for x in match])]
matches_without_prolines

[(0, 1, 2, 3),
 (4, 5, 12, 13),
 (21, 22, 28, 29),
 (30, 31, 34, 35),
 (36, 37, 42, 43),
 (44, 45, 49, 50),
 (51, 52, 59, 60),
 (61, 62, 71, 72),
 (73, 74, 82, 83),
 (84, 85, 88, 89)]

In [278]:
#get matches with overlapping nitrogens for cluster identification (this method will not get the C-terminal amino acid)
amino_acid_n = 'NCC(=O)N'
matches_n = peptide.GetSubstructMatches(Chem.MolFromSmiles(amino_acid_n),useChirality=True)
matches_n_no_prolines = [match for match in matches_n if sum([x in np.hstack(proline_matches) for x in match]) < 2]
matches_n = [match_n for match_n in matches_n]
matches_n

[(0, 1, 2, 3, 4),
 (4, 5, 12, 13, 14),
 (14, 15, 19, 20, 21),
 (21, 22, 28, 29, 30),
 (30, 31, 34, 35, 36),
 (36, 37, 42, 43, 44),
 (44, 45, 49, 50, 51),
 (51, 52, 59, 60, 61),
 (61, 62, 71, 72, 73),
 (73, 74, 82, 83, 84),
 (84, 85, 88, 89, 90)]

In [279]:
#get matches with overlapping carboynl oxygens for cluster identification (this method will not get the N-terminal amino acid)
amino_acid_c = 'O=CCNC'
matches_c = peptide.GetSubstructMatches(Chem.MolFromSmiles(amino_acid_c),useChirality=True)
matches_c_no_prolines = [match for match in matches_c if sum([x in np.hstack(proline_matches) for x in match]) < 2]
matches_c

((13, 12, 5, 4, 2),
 (20, 19, 15, 14, 12),
 (20, 19, 15, 14, 18),
 (29, 28, 22, 21, 19),
 (35, 34, 31, 30, 28),
 (43, 42, 37, 36, 34),
 (50, 49, 45, 44, 42),
 (60, 59, 52, 51, 49),
 (72, 71, 62, 61, 59),
 (83, 82, 74, 73, 71),
 (89, 88, 85, 84, 82),
 (96, 95, 91, 90, 88),
 (96, 95, 91, 90, 94))

In [280]:
#split up side chain atoms and backbone atoms
backbone_atoms = np.hstack(np.unique(np.concatenate((np.hstack(matches_n_no_prolines),np.hstack(matches_c_no_prolines),np.hstack(proline_matches),np.hstack(peptide.GetSubstructMatches(Chem.MolFromSmiles('NCC(O)=O'),useChirality=True))))))
sidechain_atoms = np.setdiff1d(np.array(range(1,peptide.GetNumAtoms())),backbone_atoms)
backbone_atoms
# sidechain_atoms

array([ 0,  1,  2,  3,  4,  5, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       28, 29, 30, 31, 34, 35, 36, 37, 42, 43, 44, 45, 49, 50, 51, 52, 59,
       60, 61, 62, 71, 72, 73, 74, 82, 83, 84, 85, 88, 89, 90, 91, 92, 93,
       94, 95, 96, 97])

In [281]:
matches_double_overlap = peptide.GetSubstructMatches(Chem.MolFromSmiles('NCC(NCC=O)=O'),useChirality=True)
matches_double_overlap = [m for m in matches_double_overlap if not any([x in sidechain_atoms for x in m])]
count = Counter(np.hstack(matches_double_overlap))
matches_double_overlap = [m for m in matches_double_overlap if not all(count[x] > 1 for x in m )]
if len(matches_double_overlap) == 2:
    n_terminus = [x for x in matches_double_overlap[0] if count[x] == 1]
    c_terminus = [x for x in matches_double_overlap[1] if count[x] == 1]
else:
    print('could not identify termini correctly')

In [282]:
#identify the termini clusters
#nterm cluster
if any([x in np.hstack(proline_matches) for x in n_terminus]): #n_terminus is a proline
    n_terminus = [np.array(proline) for proline in proline_matches if any([x in n_terminus for x in proline])][0]
else: #n_terminus is not a proline
    n_terminus = [np.array(match) for match in matches if any([x in n_terminus for x in match])][0]
n_terminus


array([0, 1, 2, 3])

In [283]:
#cterm cluster
if any([x in np.hstack(proline_matches) for x in c_terminus]):
    c_terminus = np.array(peptide.GetSubstructMatches(Chem.MolFromSmiles('CN1[C@H](C(O)=O)CCC1'),useChirality=True))[0]
else: #c terminus is not a proline
    c_terminus = np.array(peptide.GetSubstructMatches(Chem.MolFromSmiles('CNCC(=O)O'),useChirality=True))[0]
c_terminus

array([88, 90, 91, 95, 97, 96, 92, 93, 94])

In [284]:
#non-terminus prolines
prolines_non_terminal = [proline for proline in proline_matches if not any([(x in c_terminus) or (x in n_terminus) for x in proline])]
if len(prolines_non_terminal) > 0:
    prolines_not_terminal_overlap = peptide.GetSubstructMatches(Chem.MolFromSmiles('O=CC1CCCN1C'),useChirality=True)
    prolines_not_terminal_overlap = [proline for proline in prolines_not_terminal_overlap if any([x in np.hstack(prolines_non_terminal) for x in proline])]
else:
    prolines_not_terminal_overlap = []
prolines_not_terminal_overlap

[(20, 19, 15, 16, 17, 18, 14, 12)]

In [285]:
clusters = list()
[clusters.append(match) for match in matches_c_no_prolines if sum([x in c_terminus for x in match]) < 2]
clusters.append(list(c_terminus))
clusters.append(list(n_terminus))
[clusters.append(proline) for proline in prolines_not_terminal_overlap]
clusters

[(13, 12, 5, 4, 2),
 (29, 28, 22, 21, 19),
 (35, 34, 31, 30, 28),
 (43, 42, 37, 36, 34),
 (50, 49, 45, 44, 42),
 (60, 59, 52, 51, 49),
 (72, 71, 62, 61, 59),
 (83, 82, 74, 73, 71),
 (89, 88, 85, 84, 82),
 [88, 90, 91, 95, 97, 96, 92, 93, 94],
 [0, 1, 2, 3],
 (20, 19, 15, 16, 17, 18, 14, 12)]

In [286]:
##recursively identify alpha carbons, and map the atom indices that contain their side chains
c_alphas = np.array(matches)[:,1]
c_alphas_neighbors = [peptide.GetAtomWithIdx(int(x)).GetNeighbors() for x in c_alphas]

for index,c_alpha in enumerate(c_alphas):
    c_alpha_neighbors = peptide.GetAtomWithIdx(int(c_alpha)).GetNeighbors()
    print('starting c_alpha atom index: ' + str(c_alpha))
    # print('neighbor indices: ' +str([neighbor.GetIdx() for neighbor in c_alpha_neighbors]))
    # print('neighbor atomic masses: ' + str([neighbor.GetAtomicNum() for neighbor in c_alpha_neighbors]))
    # print('starting atoms: ' + str([c_alpha_neighbor.GetIdx() for c_alpha_neighbor in c_alpha_neighbors]))
    # if c_alpha in c_terminus:
    #     print('c terminus skipped: ' + str(c_alpha))
    #     continue
    if c_alpha in np.hstack(proline_matches):
        print('skipped proline')
        continue
    for neighbor in c_alpha_neighbors:
        if (neighbor.GetIdx() not in backbone_atoms) and (neighbor.GetAtomicNum() == 6):
            # print('starting side chain atom: '+str(neighbor.GetIdx()) + ', atomic num: ' + str(neighbor.GetAtomicNum()))
            side_chain_atoms = np.array([neighbor.GetIdx()])
            new_neighbors = neighbor.GetNeighbors()
            # print('starting neighbor atom(s): ' + str([new_neighbor.GetIdx() for new_neighbor in new_neighbors]) + ', atomic num(s): ' + str([new_neighbor.GetAtomicNum() for new_neighbor in new_neighbors]))
            atoms_to_examine = np.array([])
            while(len(new_neighbors) > 0):
                for new_neighbor in new_neighbors:
                    if (new_neighbor.GetIdx() not in side_chain_atoms) and (new_neighbor.GetIdx() not in backbone_atoms):
                        # print('new neighbor atom found, index: ' + str(new_neighbor.GetIdx()))
                        side_chain_atoms = np.append(side_chain_atoms,new_neighbor.GetIdx())
                        # print('side_chain_atom(s): ' + str(side_chain_atoms))
                        atoms_to_examine = np.append(atoms_to_examine,new_neighbor.GetIdx())
                        # print('atom(s) to examine: ' + str(atoms_to_examine))
                    elif new_neighbor.GetIdx()  in side_chain_atoms:
                        # print('already found this atom in side chain: ' + str(new_neighbor.GetIdx()))
                        continue
                    elif new_neighbor.GetIdx() in backbone_atoms:
                        # print('already found this atom in back bone: ' + str(new_neighbor.GetIdx()))
                        continue
                if len(atoms_to_examine) == 0:
                    # print('no new atoms to explore')
                    break
                else:
                    new_neighbors = np.hstack([peptide.GetAtomWithIdx(int(atom)).GetNeighbors() for atom in atoms_to_examine])
                    # print('new neighbors to investigate: ' + str([new_neighbor.GetIdx() for new_neighbor in new_neighbors]))
                    atoms_to_examine = np.array([])
            print('complete mapped side chain: ' + str([x for x in side_chain_atoms]))
            if not any([x in np.hstack(clusters) for x in side_chain_atoms]):
                clusters.append(np.concatenate([np.array([x for x in side_chain_atoms]),np.array([c_alpha])]))
    #neighbors are not in a consistent order

clusters
# [peptide.GetAtomWithIdx(x).GetNeighbors() for x in c_alphas]

starting c_alpha atom index: 1
starting c_alpha atom index: 5
complete mapped side chain: [6, 7, 8, 9, 10, 11, 87, 86]
starting c_alpha atom index: 15
skipped proline
starting c_alpha atom index: 22
complete mapped side chain: [23, 24, 25, 26, 27]
starting c_alpha atom index: 31
complete mapped side chain: [32, 33, 57, 56, 55, 58, 54, 53]
starting c_alpha atom index: 37
complete mapped side chain: [38, 39, 40, 41]
starting c_alpha atom index: 45
complete mapped side chain: [46, 47, 48]
starting c_alpha atom index: 52
complete mapped side chain: [53, 54, 55, 56, 57, 58, 33, 32]
starting c_alpha atom index: 62
complete mapped side chain: [63, 64, 65, 70, 66, 69, 67, 68]
starting c_alpha atom index: 74
complete mapped side chain: [75, 76, 77, 78, 79, 80, 81]
starting c_alpha atom index: 85
complete mapped side chain: [86, 87, 10, 9, 8, 11, 7, 6]
starting c_alpha atom index: 91
skipped proline


[(13, 12, 5, 4, 2),
 (29, 28, 22, 21, 19),
 (35, 34, 31, 30, 28),
 (43, 42, 37, 36, 34),
 (50, 49, 45, 44, 42),
 (60, 59, 52, 51, 49),
 (72, 71, 62, 61, 59),
 (83, 82, 74, 73, 71),
 (89, 88, 85, 84, 82),
 [88, 90, 91, 95, 97, 96, 92, 93, 94],
 [0, 1, 2, 3],
 (20, 19, 15, 16, 17, 18, 14, 12),
 array([ 6,  7,  8,  9, 10, 11, 87, 86,  5]),
 array([23, 24, 25, 26, 27, 22]),
 array([32, 33, 57, 56, 55, 58, 54, 53, 31]),
 array([38, 39, 40, 41, 37]),
 array([46, 47, 48, 45]),
 array([63, 64, 65, 70, 66, 69, 67, 68, 62]),
 array([75, 76, 77, 78, 79, 80, 81, 74])]

In [287]:
#get atom_cls
atom_cls = [[] for i in range(len(peptide.GetAtoms()))]
for i in range(len(clusters)):
    for atom in clusters[i]:
        atom_cls[atom].append(i)
atom_cls

[[10],
 [10],
 [0, 10],
 [10],
 [0],
 [0, 12],
 [12],
 [12],
 [12],
 [12],
 [12],
 [12],
 [0, 11],
 [0],
 [11],
 [11],
 [11],
 [11],
 [11],
 [1, 11],
 [11],
 [1],
 [1, 13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [1, 2],
 [1],
 [2],
 [2, 14],
 [14],
 [14],
 [2, 3],
 [2],
 [3],
 [3, 15],
 [15],
 [15],
 [15],
 [15],
 [3, 4],
 [3],
 [4],
 [4, 16],
 [16],
 [16],
 [16],
 [4, 5],
 [4],
 [5],
 [5],
 [14],
 [14],
 [14],
 [14],
 [14],
 [14],
 [5, 6],
 [5],
 [6],
 [6, 17],
 [17],
 [17],
 [17],
 [17],
 [17],
 [17],
 [17],
 [17],
 [6, 7],
 [6],
 [7],
 [7, 18],
 [18],
 [18],
 [18],
 [18],
 [18],
 [18],
 [18],
 [7, 8],
 [7],
 [8],
 [8],
 [12],
 [12],
 [8, 9],
 [8],
 [9],
 [9],
 [9],
 [9],
 [9],
 [9],
 [9],
 [9]]

array([ 99, 101, 102, 106, 108, 107, 103, 104, 105])